In [1]:
import pandas as pd
from datasets import load_dataset
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import DistilBertTokenizer

In [2]:
dataset = load_dataset("fancyzhx/yelp_polarity")

README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [3]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

In [4]:
train_df.head()

,text,label
0,"Unfortunately, the frustration of being Dr. Go...",0
1,Been going to Dr. Goldberg for over 10 years. ...,1
2,I don't know what Dr. Goldberg was like before...,0
3,I'm writing this review to give you a heads up...,0
4,All the food is great here. But the best thing...,1


In [5]:
test_df.head()

,text,label
0,"Contrary to other reviews, I have zero complai...",1
1,Last summer I had an appointment to get new ti...,0
2,"Friendly staff, same starbucks fair you get an...",1
3,The food is good. Unfortunately the service is...,0
4,Even when we didn't have a car Filene's Baseme...,1


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    560000 non-null  object
 1   label   560000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.5+ MB


In [7]:
train_df.isnull().sum()

text     0
label    0
dtype: int64

In [8]:
train_df.sample(5)['text'].values

array(['If you are in a hurry or think you are going to get in and out, you will be very disappointed. I waited an hour and a half to take a drug test for a new job. When I asked what is going on the lady at the front desk told me that there was a medical emergency and I would have to wait longer.( she is not the doctor) again I was only here to pee in a cup. Horrible service!!!!!',
       'The mini putt here is not so great.  None of the water features are filled. The greens and scenery are pretty run down too. I liked that it was open late. If you want someplace a little more entertaining try the castles & coasters Northwest of here.',
       'My wife and I went there for lunch about 2:30pm today, 10/28/14. \\n\\nThe service was slow, the iced tea was so weak it looked like water for 2.50 a glass. My food was cold and looked like it had been sitting for hours. I can understand that this was an off hour, but I expect good quality food and service all the time.\\n\\nWhat a disappointme

In [9]:
def clean_text(text):
    text = text.replace('\\n', ' ') 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\\"', '"')
    text = text.strip()     
    return text
train_df['text'] = train_df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

In [10]:
train_df.duplicated(subset='text').sum()

49

In [11]:
train_df = train_df.drop_duplicates(subset='text').reset_index(drop=True)

In [12]:
train_df.duplicated(subset='text').sum()

0

In [13]:
test_df.duplicated(subset='text').sum()

0

In [14]:

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [15]:

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = dataset['test']

In [16]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=256)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [17]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/503955 [00:00<?, ? examples/s]

Map:   0%|          | 0/55996 [00:00<?, ? examples/s]

Map:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [18]:
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_val = tokenized_val.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

In [19]:
tokenized_train

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 503955
})

In [20]:
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_val = tokenized_val.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])